In [52]:
import pandas as pd
import os
from sklearn.ensemble import RandomForestClassifier
import numpy as np

def predicted_bugNum_vs_true_bugNum(Y_test_predict,Y_test_bugs):
    n = 0
    bugrowcount = 0
    for i in range(0,len(Y_test_predict)):
        if(Y_test_bugs[i]!=False):
            bugrowcount += 1
            if(Y_test_predict[i]==Y_test_bugs[i]):
                n=n+1
#    print(n/bugrowcount,n,bugrowcount)
    return n/bugrowcount,n

def load_one_version_method_matrix(matrix_path):
    print(matrix_path)
    return pd.read_csv(matrix_path)

def get_selected_rows(columncombation,X_data):
    flag = False
    for i in columncombation:
        if(flag == False):
            c = X_data[:,i].T
            flag = True
        else:
            c = np.vstack((c,X_data[:,i].T))
    X_train_selected = c.T 
    return X_train_selected

def classification_binary(csvpath,startColumnName,
                          endColumnName,isdefects,importance_result,
                         columncombation_j = None):

    filePath = "AlltheMatrix/"

    filePathNowusing = filePath + csvpath

    changed_matrix = load_one_version_method_matrix(filePathNowusing)
    print(f'lens: {len(changed_matrix.columns.values)}')
    matrixlen = int(len(changed_matrix))
    startindex = 0
    endindex = 0
    for i, column in enumerate(changed_matrix.columns.values):
        if(column == startColumnName):
            startindex = i
        elif (column == endColumnName):
            endindex = i
    print(f'start:{startindex},end:{endindex}') 
    target = changed_matrix[endColumnName].values
    data = changed_matrix.values[:,startindex+1:endindex]
    print(target.shape)
    print(data.shape)

    #shuffleIndex
    import numpy as np
    shuffle_index = np.random.permutation(matrixlen)
    target, data = target[shuffle_index], data[shuffle_index]

    #shuffleIndex
    import numpy as np
    shuffle_index = np.random.permutation(matrixlen)
    target, data = target[shuffle_index], data[shuffle_index]
    print(type(data))
    #split the train set and the test set
    X_train = data[:round(0.7*matrixlen)]
    X_test = data[round(0.7*matrixlen):]
    Y_train = target[:round(0.7*matrixlen)]
    Y_test = target[round(0.7*matrixlen):]
    print(X_train,Y_train.shape)
    #train a binary classifier
    Y_train_bugs = (Y_train > isdefects)
    Y_test_bugs = (Y_test > isdefects)
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import cross_val_score

    if(columncombation_j!=None):
        print("####################################")
        X_train = get_selected_rows(columncombation_j,X_train)
        X_test = get_selected_rows(columncombation_j,X_test)

    forest_clf = RandomForestClassifier(n_estimators = 10)
    forest_clf.fit(X_train,Y_train_bugs)

    cross_result = cross_val_score(forest_clf, X_train, Y_train_bugs, cv = 5, scoring = 'accuracy')
    Y_forest_predict = forest_clf.predict(X_train)

    from sklearn.metrics import precision_score, recall_score
    forst_precision = precision_score(Y_train_bugs, Y_forest_predict)
    forst_recall = recall_score(Y_train_bugs, Y_forest_predict)
    print(f'Forest_accurancy is {forst_precision}')
    print(f'Forest_recall is {forst_recall}')
    print(cross_result)
    #    print(X_train_selected)
    Y_test_predict = forest_clf.predict(X_test)

    findingRate = predicted_bugNum_vs_true_bugNum(Y_test_predict,Y_test_bugs)
    print(f'fingdingrate = {findingRate}')
    
    # get the importance of each column
    importances=forest_clf.feature_importances_
    print('the importance of the each column：\n',importances)
    indices = np.argsort(importances)[::-1]
    print('the descend order for importance of the column：\n',indices)
    most_import = indices[:10]
    print(f'most importtant columns:{most_import}')
    print(X_train[:,most_import])
    

    columnlist = np.arange(0,endindex-startindex-1,1)
    for i in columnlist:
        if i in most_import:
            importance_result[i] +=1

#     import csv
#     # have to change the output file name
#     with open('PredictedAlltheMatrix/importance_result.csv','a+') as f:
#         csv_write = csv.writer(f)
#         data_row = [most_import,forst_precision,forst_recall,findingRate,np.mean(cross_result)]
#         csv_write.writerow(data_row)

In [53]:
importance_result = {}
columnlist = np.arange(0,36,1)
for i in columnlist:
    importance_result[i] = 0
for i in range(100):
    classification_binary('all_data.csv','numberOfVersionsUntil','bugs',0,importance_result)
importance_result

AlltheMatrix/all_data.csv
lens: 49
start:7,end:44
(1497,)
(1497, 36)
<class 'numpy.ndarray'>
[[2 0 9 ... 14 49 25]
 [0 0 4 ... 2 6 4]
 [0 0 4 ... 13 45 29]
 ...
 [0 0 5 ... 5 31 10]
 [1 0 5 ... 2 43 25]
 [1 0 5 ... 6 82 25]] (1048,)
Forest_accurancy is 1.0
Forest_recall is 0.9230769230769231
[0.85781991 0.86190476 0.86124402 0.86602871 0.86602871]
fingdingrate = (0.16981132075471697, 9)
the importance of the each column：
 [0.00503879 0.00198035 0.01109849 0.0235456  0.02945482 0.02555117
 0.02782642 0.02724471 0.02918161 0.02318053 0.02699936 0.02226939
 0.0334552  0.0239159  0.08566515 0.07405875 0.05123134 0.05246331
 0.06343442 0.01819742 0.01847726 0.01383874 0.02161299 0.023842
 0.00615067 0.02829434 0.00659506 0.04141088 0.02050308 0.03772588
 0.01908834 0.01747539 0.00728091 0.02502498 0.03292836 0.02395839]
the descend order for importance of the column：
 [14 15 18 17 16 27 29 12 34  4  8 25  6  7 10  5 33 35 13 23  3  9 11 22
 28 30 20 19 31 21  2 32 26 24  0  1]
most importta

Forest_accurancy is 0.9916666666666667
Forest_recall is 0.85
[0.87619048 0.88095238 0.87619048 0.88038278 0.8708134 ]
fingdingrate = (0.18840579710144928, 13)
the importance of the each column：
 [0.01317992 0.00553002 0.01345495 0.04961575 0.03337438 0.01762392
 0.03308488 0.03868542 0.02902267 0.03353491 0.02640859 0.03490919
 0.02731733 0.02870841 0.04349244 0.04658659 0.0346223  0.06596287
 0.03706962 0.02112    0.01415472 0.01556361 0.04366548 0.00742202
 0.00520757 0.02900948 0.01850629 0.02923326 0.01993111 0.03291273
 0.01242404 0.02496474 0.00646034 0.04569725 0.04117488 0.02036832]
the descend order for importance of the column：
 [17  3 15 33 22 14 34  7 18 11 16  9  4  6 29 27  8 25 13 12 10 31 19 35
 28 26  5 21 20  2  0 30 23 32  1 24]
most importtant columns:[17  3 15 33 22 14 34  7 18 11]
[[0.28814 78 0.0128268 ... 13 0.0346645 3.6428599999999998]
 [0.0609203 1 7.79e-05 ... 1 0.0309529 0.0]
 [0.314288 182 0.0364065 ... 69 0.0408781 1.26316]
 ...
 [0.138856 414 0.0241977 .

Forest_accurancy is 1.0
Forest_recall is 0.910958904109589
[0.87203791 0.86190476 0.83253589 0.84688995 0.86124402]
fingdingrate = (0.19047619047619047, 12)
the importance of the each column：
 [0.00631862 0.0099954  0.01793129 0.03316789 0.02750038 0.03009407
 0.06486916 0.02553862 0.02407487 0.02768608 0.03534679 0.02866868
 0.02333004 0.0296507  0.03172607 0.03238263 0.06634596 0.07134678
 0.04946684 0.02521939 0.01755891 0.01210468 0.02529041 0.027101
 0.0117361  0.01720928 0.00740507 0.04045613 0.01911256 0.02501887
 0.018356   0.01062902 0.01147487 0.03955205 0.03212061 0.02421418]
the descend order for importance of the column：
 [17 16  6 18 27 33 10  3 15 34 14  5 13 11  9  4 23  7 22 19 29 35  8 12
 28 30  2 20 25 21 24 32 31  1 26  0]
most importtant columns:[17 16  6 18 27 33 10  3 15 34]
[[0.16933399999999998 0.0163241 30 ... 20 0.0038316 35]
 [0.179524 0.0220027 49 ... 33 0.015080700000000002 46]
 [0.23163699999999998 0.0225766 258 ... 214 0.0255641 68]
 ...
 [0.30794099999

Forest_accurancy is 1.0
Forest_recall is 0.8733333333333333
[0.85714286 0.85238095 0.86666667 0.8708134  0.87559809]
fingdingrate = (0.1864406779661017, 11)
the importance of the each column：
 [0.00228993 0.00701354 0.02197726 0.04328604 0.02452577 0.02191732
 0.02143127 0.02507992 0.0296728  0.03479651 0.02523689 0.03057655
 0.02766416 0.03544201 0.05797966 0.04831161 0.07062649 0.05963221
 0.03299983 0.0192218  0.01630931 0.01018111 0.02393107 0.0176107
 0.0055266  0.02131914 0.01153023 0.07861811 0.01857759 0.03161249
 0.0152337  0.02091317 0.00434188 0.02629826 0.02828364 0.03003141]
the descend order for importance of the column：
 [27 16 17 14 15  3 13  9 18 29 11 35  8 34 12 33 10  7  4 22  2  5  6 25
 31 19 28 23 20 30 26 21  1 24 32  0]
most importtant columns:[27 16 17 14 15  3 13  9 18 29]
[[36 0.0172857 0.184482 ... 5 0.0344522 0]
 [49 0.0100966 0.0617154 ... -2 0.0339284 0]
 [97 0.028992400000000005 0.40219699999999997 ... 96 0.0347774 41]
 ...
 [525 0.08934260000000001 0.7

Forest_accurancy is 1.0
Forest_recall is 0.9205298013245033
[0.86729858 0.87142857 0.84210526 0.85167464 0.86602871]
fingdingrate = (0.13793103448275862, 8)
the importance of the each column：
 [0.00671356 0.00330479 0.01099091 0.03177307 0.01953436 0.02824468
 0.04104387 0.02481524 0.03073409 0.02456928 0.01782374 0.02540198
 0.04802552 0.04125521 0.05683174 0.04145719 0.04451117 0.04825332
 0.03971408 0.01876499 0.01538734 0.01578239 0.02991056 0.02712043
 0.00746367 0.01745677 0.00926456 0.0500307  0.02737374 0.03375633
 0.03094972 0.02267192 0.00773571 0.02606886 0.04757527 0.02768924]
the descend order for importance of the column：
 [14 27 17 12 34 16 15 13  6 18 29  3 30  8 22  5 35 28 23 33 11  7  9 31
  4 19 10 25 21 20  2 26 32 24  0  1]
most importtant columns:[14 27 17 12 34 16 15 13  6 18]
[[11.0669 27 0.28965799999999997 ... 155.97799999999998 72
  0.04309330000000001]
 [1.76682 7 0.0617154 ... 24.7143 2 0.0339284]
 [10.7594 21 0.282301 ... 190.706 44 0.043071]
 ...
 [11.50

...

Forest_accurancy is 0.9927007299270073
Forest_recall is 0.8717948717948718
[0.86729858 0.83809524 0.83732057 0.85167464 0.83253589]
fingdingrate = (0.09433962264150944, 5)
the importance of the each column：
 [0.00585187 0.00650316 0.02035985 0.05243698 0.0242612  0.02239203
 0.02769542 0.0311337  0.03291989 0.03941893 0.02425257 0.02244748
 0.04285519 0.03892603 0.03100598 0.05486298 0.03715899 0.05335881
 0.06386218 0.0316828  0.01555116 0.01720381 0.03476764 0.01434753
 0.00674317 0.03051683 0.00344898 0.03231861 0.02276848 0.02234987
 0.02572858 0.0177391  0.01274123 0.02856228 0.02003179 0.03179486]
the descend order for importance of the column：
 [18 15 17  3 12  9 13 16 22  8 27 35 19  7 14 25 33  6 30  4 10 28 11  5
 29  2 34 31 21 20 23 32 24  1  0 26]
most importtant columns:[18 15 17  3 12  9 13 16 22  8]
[[0.374889 0.782287 4.84923 ... 0.18529 21 30.4583]
 [0.051703099999999995 0.153059 0.271362 ... 0.0295029 7 26.25]
 [0.0463837 0.00704989 0.138856 ... 0.018583199999999998 

Forest_accurancy is 1.0
Forest_recall is 0.8843537414965986
[0.85781991 0.87619048 0.85645933 0.86124402 0.85167464]
fingdingrate = (0.20967741935483872, 13)
the importance of the each column：
 [0.00959151 0.00157988 0.01170778 0.03617046 0.03977589 0.03233533
 0.05161692 0.02447945 0.01906601 0.02884981 0.04340327 0.02431635
 0.02378595 0.02991879 0.02078722 0.03671215 0.05708499 0.028232
 0.04308052 0.01597133 0.01704029 0.01587026 0.03348247 0.02562951
 0.00818399 0.02283262 0.00681645 0.0433073  0.02899306 0.03731577
 0.03240886 0.01915609 0.00902296 0.03510102 0.03802034 0.04835338]
the descend order for importance of the column：
 [16  6 35 10 27 18  4 34 29 15  3 33 22 30  5 13 28  9 17 23  7 11 12 25
 14 31  8 20 19 21  2  0 32 24 26  1]
most importtant columns:[16  6 35 10 27 18  4 34 29 15]
[[0.018996799999999998 3 5 ... 5 10 0.0158563]
 [0.025235 12 5 ... 20 10 0.00674961]
 [0.0603233 397 11 ... 28 0 0.177304]
 ...
 [0.0337324 570 9 ... 25 15 0.0976143]
 [0.0137916 32 9 ... 1

Forest_accurancy is 1.0
Forest_recall is 0.9090909090909091
[0.85714286 0.86666667 0.88095238 0.85238095 0.87019231]
fingdingrate = (0.14545454545454545, 8)
the importance of the each column：
 [0.00701114 0.00276685 0.00968905 0.03926378 0.02522474 0.03305276
 0.05332321 0.03800302 0.03034837 0.03629365 0.01620552 0.03026357
 0.02690787 0.03710507 0.03259236 0.0486341  0.04471764 0.07362572
 0.06463178 0.02431271 0.01461192 0.02113482 0.02026242 0.01873489
 0.00867955 0.01960433 0.01269725 0.02802236 0.01557547 0.03027213
 0.0212318  0.02159162 0.00067567 0.0237255  0.04778556 0.0214218 ]
the descend order for importance of the column：
 [17 18  6 15 34 16  3  7 13  9  5 14  8 29 11 27 12  4 19 33 31 35 30 21
 22 25 23 10 28 20 26  2 24  0  1 32]
most importtant columns:[17 18  6 15 34 16  3  7 13  9]
[[0.09828619999999999 0.0354147 45 ... 41 66.1317 6]
 [0.16933399999999998 0.03435680000000001 27 ... 8 176.245 12]
 [4.6253400000000005 0.33361399999999997 206 ... 52 159.309 209]
 ...
 [

Forest_accurancy is 0.9926470588235294
Forest_recall is 0.9060402684563759
[0.83809524 0.84285714 0.84761905 0.85238095 0.86057692]
fingdingrate = (0.15, 9)
the importance of the each column：
 [0.00253998 0.00613003 0.01626633 0.0477217  0.03308633 0.02174668
 0.02719147 0.02235993 0.02063064 0.03261823 0.01725677 0.01922377
 0.03131045 0.03236833 0.04795218 0.05159518 0.06051358 0.06406203
 0.04589228 0.01364887 0.02379991 0.01134882 0.0393602  0.01879531
 0.0098577  0.01620325 0.00937324 0.06953731 0.01358982 0.04266907
 0.0183545  0.01776637 0.00841086 0.02684872 0.02848497 0.03148519]
the descend order for importance of the column：
 [27 17 16 15 14  3 18 29 22  4  9 13 35 12 34  6 33 20  7  5  8 11 23 30
 31 10  2 25 19 28 21 24 26 32  1  0]
most importtant columns:[27 17 16 15 14  3 18 29 22  4]
[[164 0.0 0.0 ... 6 12 4]
 [174 0.23565300000000003 0.0200703 ... 29 17 31]
 [416 0.563558 0.0487781 ... 11 25 232]
 ...
 [38 0.0617154 0.0100966 ... 0 3 8]
 [6 0.0617154 0.0100966 ... 14 

{0: 0,
 1: 0,
 2: 0,
 3: 51,
 4: 14,
 5: 18,
 6: 58,
 7: 34,
 8: 17,
 9: 30,
 10: 8,
 11: 16,
 12: 42,
 13: 18,
 14: 68,
 15: 88,
 16: 89,
 17: 86,
 18: 93,
 19: 6,
 20: 3,
 21: 0,
 22: 21,
 23: 4,
 24: 0,
 25: 4,
 26: 0,
 27: 70,
 28: 2,
 29: 29,
 30: 1,
 31: 1,
 32: 0,
 33: 32,
 34: 61,
 35: 36}

In [56]:
sorted(importance_result.items(),key = lambda item:item[1])[::-1]

[(18, 93),
 (16, 89),
 (15, 88),
 (17, 86),
 (27, 70),
 (14, 68),
 (34, 61),
 (6, 58),
 (3, 51),
 (12, 42),
 (35, 36),
 (7, 34),
 (33, 32),
 (9, 30),
 (29, 29),
 (22, 21),
 (13, 18),
 (5, 18),
 (8, 17),
 (11, 16),
 (4, 14),
 (10, 8),
 (19, 6),
 (25, 4),
 (23, 4),
 (20, 3),
 (28, 2),
 (31, 1),
 (30, 1),
 (32, 0),
 (26, 0),
 (24, 0),
 (21, 0),
 (2, 0),
 (1, 0),
 (0, 0)]

In [48]:
topTenImportantColumn = sorted(importance_result.items(),key = lambda item:item[1])[-6:]

In [54]:
topTenImportantColumn

[(27, 14), (14, 15), (18, 23), (15, 24), (16, 25), (17, 25)]

In [50]:
tup = ()
for i,one in enumerate(topTenImportantColumn):
    tup += (one[0],)
print(tup)
classification_binary('all_data.csv','numberOfVersionsUntil',
                      'bugs',0,importance_result,tup)


(27, 14, 18, 15, 16, 17)
AlltheMatrix/all_data.csv
lens: 49
start:7,end:44
(1497,)
(1497, 36)
<class 'numpy.ndarray'>
[[1 0 5 ... 3 12 12]
 [0 0 6 ... 5 35 17]
 [0 0 5 ... 1 19 19]
 ...
 [0 0 2 ... 1 5 1]
 [0 0 3 ... 8 30 14]
 [0 0 2 ... 5 12 9]] (1048,)
####################################
Forest_accurancy is 1.0
Forest_recall is 0.8775510204081632
[0.86729858 0.89047619 0.86124402 0.8708134  0.86124402]
fingdingrate = (0.11290322580645161, 7)
the importance of the each column：
 [0.17954621 0.16393566 0.16099739 0.16097185 0.16445011 0.17009879]
the descend order for importance of the column：
 [0 5 4 1 2 3]
most importtant columns:[0 5 4 1 2 3]
[[38 0.23947399999999996 0.0210053 9.315489999999999 0.03526780000000001
  0.00598493]
 [71 0.378897 0.032213099999999995 14.7948 0.0440326 0.05622080000000001]
 [37 0.300266 0.0233267 12.2006 0.034587900000000005 0.0154827]
 ...
 [14 0.0617154 0.0100966 1.76682 0.0339284 0.000249418]
 [53 0.10365 0.0124286 3.54759 0.031312900000000005 0.001531